In [29]:
from pathlib import Path
import pandas as pd
from datetime import datetime
import ast

# 1. Define the project root directory
project_root = Path.cwd().parent  # Assuming the notebook is in 'notebooks/', project_root is 'Machine-Learning_Project/'

# 2. Define the paths to the Excel files
profiles_path = project_root / "Tables" / "raw_tables" / "dados_sem_listas.xlsx"
positions_path = project_root / "Tables" / "raw_tables" / "dados_positions.xlsx"

# 3. Read the Excel files
profiles = pd.read_excel(profiles_path)
positions = pd.read_excel(positions_path)

# 4. Parse the 'timePeriod' column in positions
def parse_time_period(time_period_str):
    try:
        return ast.literal_eval(time_period_str)
    except (ValueError, SyntaxError) as e:
        print(f"Erro ao analisar timePeriod: {e}")
        return {}

positions['timePeriod_parsed'] = positions['timePeriod'].apply(parse_time_period)

# 5. Extract start_date and end_date from 'timePeriod_parsed'
def extract_dates(time_period):
    # Extract start date components
    start_info = time_period.get('startDate', {})
    start_year = start_info.get('year')
    start_month = start_info.get('month')
    
    if start_year is None or start_month is None:
        start_date = pd.NaT  # Not a Time (missing start date)
    else:
        start_date = datetime(year=start_year, month=start_month, day=1)
    
    # Extract end date components
    end_info = time_period.get('endDate', {})
    end_year = end_info.get('year')
    end_month = end_info.get('month')
    
    if end_year and end_month:
        # Define end_date as the last day of the end month
        end_date = datetime(year=end_year, month=end_month, day=1) + pd.offsets.MonthEnd(1)
    else:
        # If no endDate, set to today's date
        end_date = datetime.today()
    
    return pd.Series({'start_date': start_date, 'end_date': end_date})

# Define the mapping for standardizing 'industryName'
industry_mapping = {
    'Investment Banking': 'Banking',
    'Management Consulting': 'Business Consulting and Services',
    'IT Services and IT Consulting': 'Information Technology',
    'Information Technology & Services': 'Information Technology',
    'Technology, Information and Internet': 'Information Technology',
    'Computer Software': 'Information Technology',
    'Higher Education': 'Education & Research',
    'Research Services': 'Education & Research',
    'Education': 'Education & Research',
    'Education Administration Programs': 'Education & Research'
}

# Apply the mapping to the 'industryName' column
profiles['industryName'] = profiles['industryName'].replace(industry_mapping)


positions[['start_date', 'end_date']] = positions['timePeriod_parsed'].apply(extract_dates)

# 6. Compute experience in days
positions['experience'] = (positions['end_date'] - positions['start_date']).dt.days

# 7. Aggregate experience by summing days per 'id'
experience_per_id = positions.groupby('id')['experience'].sum().reset_index()

# 8. Merge the aggregated experience with profiles on 'id'
final_df = pd.merge(profiles, experience_per_id, on='id', how='left')

# 9. Handle 'Unnamed: 0' column
if 'Unnamed: 0' not in final_df.columns:
    # If 'Unnamed: 0' is not a column, reset the index to include it
    final_df = final_df.reset_index().rename(columns={'index': 'Unnamed: 0'})

# 10. Select the desired columns
final_df = final_df[['Unnamed: 0', 'id', 'industryName', 'followersCount', 'experience']]

# 11. Remove rows where 'experience' is NA
final_df = final_df.dropna(subset=['experience'])

# 12. Convert data types
final_df['Unnamed: 0'] = final_df['Unnamed: 0'].astype(int)
final_df['id'] = final_df['id'].astype(int)
final_df['industryName'] = final_df['industryName'].astype(str)
final_df['followersCount'] = final_df['followersCount'].astype(int)
final_df['experience'] = final_df['experience'].astype(float)  # Use int if no decimal needed

# 13. Display the final DataFrame
print("Final DataFrame:")
display(final_df)


Erro ao analisar timePeriod: malformed node or string: nan
Erro ao analisar timePeriod: malformed node or string: nan
Erro ao analisar timePeriod: malformed node or string: nan
Erro ao analisar timePeriod: malformed node or string: nan
Erro ao analisar timePeriod: malformed node or string: nan
Erro ao analisar timePeriod: malformed node or string: nan
Erro ao analisar timePeriod: malformed node or string: nan
Erro ao analisar timePeriod: malformed node or string: nan
Erro ao analisar timePeriod: malformed node or string: nan
Erro ao analisar timePeriod: malformed node or string: nan
Erro ao analisar timePeriod: malformed node or string: nan
Erro ao analisar timePeriod: malformed node or string: nan
Final DataFrame:


,Unnamed: 0,id,industryName,followersCount,experience
0,0,43695134,Banking,1476,2580.0
1,1,154315745,Banking,666,4925.0
2,2,643247,Banking,802,9063.0
3,3,107217342,Banking,579,4714.0
4,4,203090080,Advertising Services,5350,3951.0
...,...,...,...,...,...
1228,1228,47938160,Education & Research,1619,6596.0
1229,1229,76962470,Personal Care Product Manufacturing,8363,3064.0
1230,1230,888929,Oil and Gas,2622,5896.0
1231,1231,33991350,Banking,1415,14168.0
